In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [10]:
%%time 
train_data = pd.read_csv("data/ohe_train_recipes_v2.csv",index_col="id")
# test_data = pd.read_csv("data/ohe_test_recipes_v2.csv",index_col="id")

CPU times: user 9.82 s, sys: 1.04 s, total: 10.9 s
Wall time: 11.2 s


In [72]:
train_data['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [7]:
train_data.head(2)

,1% buttermilk,1% chocolate milk,1% cottage cheese,1% milk,"2 1/2 to 3 lb. chicken, cut into serving pieces",2% cottage cheese,2% low fat cheddar chees,2% lowfat greek yogurt,2% milk mozzarella cheese,2% reduced-fat milk,...,yuzu,yuzu juice,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini,zucchini blossoms,cuisine
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spanish
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mexican


In [47]:
X=train_data.drop(['cuisine'],axis=1)
response=train_data['cuisine']

In [48]:
X_train,X_validate,y_train,y_validate=train_test_split(X,response,test_size=0.5,random_state=22)

In [49]:
X_train.shape[0], X_validate.shape[0], X_validate.shape[0]/(X_validate.shape[0]+X_train.shape[0])

(19887, 19887, 0.5)

In [50]:
%%time
lr=LogisticRegression(C=1)
lr.fit(X_train,y_train)

CPU times: user 5min 17s, sys: 2.26 s, total: 5min 19s
Wall time: 45.2 s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1)

In [51]:
lr.score(X_validate,y_validate), lr.score(X_train,y_train)

(0.765676069794338, 0.9033036657112686)

In [53]:
lr.classes_

array(['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino',
       'french', 'greek', 'indian', 'irish', 'italian', 'jamaican',
       'japanese', 'korean', 'mexican', 'moroccan', 'russian',
       'southern_us', 'spanish', 'thai', 'vietnamese'], dtype=object)

In [55]:
LR_coeffs_df = pd.DataFrame(data=lr.coef_, index=lr.classes_, columns=X.columns)

In [61]:
LR_coeffs_df.loc['italian',:].sort_values()

sweetened condensed milk   -1.710693
cajun seasoning            -1.691252
saffron                    -1.675285
crawfish                   -1.566172
edamame                    -1.458014
                              ...   
pesto                       2.288380
marsala wine                2.315370
mascarpone                  2.578547
arborio rice                2.621228
polenta                     2.972773
Name: italian, Length: 6215, dtype: float64

In [62]:
LR_coeffs_df.loc['thai',:].sort_values()

cinnamon                   -1.222740
butter                     -1.220154
pepper                     -1.071114
unsalted butter            -1.047202
toasted sesame seeds       -0.904651
                              ...   
chunky peanut butter        2.556025
fish sauce                  2.603484
unsweetened coconut milk    2.723455
sweet chili sauce           2.949666
Thai red curry paste        3.266704
Name: thai, Length: 6215, dtype: float64

In [67]:
LR_coeffs_df.loc['french',:].sort_values(ascending=False).head(20)

gruyere cheese           2.849527
grated Gruyère cheese    2.389507
herbes de provence       2.241238
swiss cheese             2.026017
haricots verts           1.772974
asparagus spears         1.632134
chocolate morsels        1.627251
fleur de sel             1.587870
tarragon leaves          1.573579
Niçoise olives           1.558507
brie cheese              1.476919
cognac                   1.458828
lime rind                1.449055
crème fraîche            1.438466
crepes                   1.421718
leeks                    1.418907
pie dough                1.408503
armagnac                 1.400984
vanilla beans            1.398356
tarragon                 1.387922
Name: french, dtype: float64

In [69]:
LR_coeffs_df.loc['southern_us',:].sort_values(ascending=False).head(20)

black-eyed peas           3.799999
quickcooking grits        3.164961
grits                     3.010306
collard greens            3.006020
bourbon whiskey           2.883892
peaches                   2.717806
key lime juice            2.625164
green tomatoes            2.560969
refrigerated piecrusts    2.559063
white cornmeal            2.258886
self rising flour         2.202069
buttermilk                2.189468
country ham               2.186929
cornbread                 2.166908
barbecue sauce            2.130010
vidalia onion             2.081101
vanilla wafers            1.944843
peach nectar              1.938728
tea bags                  1.931844
crawfish                  1.890391
Name: southern_us, dtype: float64

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
scaler = StandardScaler()
X_train_trans = scaler.fit_transform(X_train)

In [30]:
X_validate_trans = scaler.transform(X_validate)

In [31]:
%%time
lr=LogisticRegression()
lr.fit(X_train_trans,y_train)

CPU times: user 6min, sys: 3.27 s, total: 6min 3s
Wall time: 52.6 s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
lr.score(X_validate_trans,y_validate), lr.score(X_train_trans,y_train)

(0.6817518982249711, 0.9927590888520139)

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
parameters = {'C':[0.05,0.1,0.5,1,2]}
lr=LogisticRegression()

In [36]:
clf = GridSearchCV(lr, parameters)

In [37]:
clf.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

KeyboardInterrupt: 

In [11]:
clustering = KMeans(init='k-means++', n_clusters=20)


In [12]:
clustering.fit(X_train)

KMeans(n_clusters=20)

In [17]:
clusters_X_train    = pd.Series(clustering.predict(X_train),   name="AutoCluster",index=X_train.index)
clusters_X_validate = pd.Series(clustering.predict(X_validate),name="AutoCluster",index=X_validate.index)

In [19]:
X_train_w_clusters = X_train.merge(clusters_X_train, left_index=True, right_index=True)
X_validate_w_clusters = X_validate.merge(clusters_X_validate, left_index=True, right_index=True)

In [20]:
%%time
lr=LogisticRegression()
lr.fit(X_train_w_clusters,y_train)

CPU times: user 11min 42s, sys: 3.91 s, total: 11min 45s
Wall time: 1min 49s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
%%time
lr.score(X_train_w_clusters,y_train), lr.score(X_validate_w_clusters,y_validate)

CPU times: user 7.4 s, sys: 1.37 s, total: 8.77 s
Wall time: 2.51 s


(0.7753809020968472, 0.7330416855232061)

In [3]:
%%time 
train_data_tfidf = pd.read_csv("data/tfidf_train_recipes_v2.csv",index_col="id")


CPU times: user 14.9 s, sys: 1.49 s, total: 16.4 s
Wall time: 17.3 s


In [4]:
from sklearn.decomposition import PCA

In [5]:
train_data_tfidf.shape

(39774, 6215)

In [11]:
train_data.loc[train_data_tfidf.index,'cuisine']

id
0            spanish
1            mexican
2             french
3            chinese
4            italian
            ...     
49712         indian
49713    southern_us
49714          irish
49716         indian
49717        mexican
Name: cuisine, Length: 39774, dtype: object

In [12]:
X_train_tfidf,X_val_tfidf,y_train_tfidf,y_val_tfidf=train_test_split(train_data_tfidf,train_data.loc[train_data_tfidf.index,'cuisine'],
                                                                     test_size=0.33,random_state=22)

In [13]:
pca = PCA(n_components=3000)
pca.fit(X_train_tfidf)
X_red_train_tfidf = pca.transform(X_train_tfidf)
X_red_val_tfidf = pca.transform(X_val_tfidf)

In [20]:
y_red_train = train_data.loc[X_train_tfidf.index,'cuisine']
y_red_val   = train_data.loc[X_val_tfidf.index,  'cuisine']

In [21]:
y_red_train.shape, X_red_train_tfidf.shape

((26648,), (26648, 3000))

In [22]:
%%time
lr=LogisticRegression()
lr.fit(X_red_train_tfidf,y_red_train)

CPU times: user 3min 20s, sys: 1.59 s, total: 3min 22s
Wall time: 27.3 s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [23]:
%%time
lr.score(X_red_train_tfidf,y_red_train), lr.score(X_red_val_tfidf,y_red_val)

CPU times: user 1.88 s, sys: 215 ms, total: 2.09 s
Wall time: 304 ms


(0.8289927949564695, 0.765122657321347)

In [24]:
## try iterations of training logistic regression with different PCA dimensions 


In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
parameters = {'C':[0.05,0.1,0.5,1,2]}
lr=LogisticRegression()

In [32]:
clf = GridSearchCV(lr, parameters,verbose=2,cv=3)

In [31]:
X_red_val_tfidf.shape

(13126, 3000)

In [33]:
clf.fit(X_red_val_tfidf,y_red_val)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END .............................................C=0.05; total time=   5.1s
[CV] END .............................................C=0.05; total time=   5.4s
[CV] END .............................................C=0.05; total time=   5.0s
[CV] END ..............................................C=0.1; total time=   6.4s
[CV] END ..............................................C=0.1; total time=   6.2s
[CV] END ..............................................C=0.1; total time=   5.4s
[CV] END ..............................................C=0.5; total time=  10.3s
[CV] END ..............................................C=0.5; total time=   9.6s
[CV] END ..............................................C=0.5; total time=   9.0s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................................C=1; total time=   9.9s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................................C=1; total time=  11.3s


...

[CV] END ................................................C=2; total time=   9.0s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................................C=2; total time=   9.4s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [0.05, 0.1, 0.5, 1, 2]}, verbose=2)

In [36]:
clf.best_score_, clf.best_params_

(0.7329724558196222, {'C': 2})

In [38]:
X_val_tfidf.shape

(13126, 6215)

In [40]:
clf.best_estimator_.score(X_red_train_tfidf, y_red_train)

0.7599069348543981

In [45]:
clf.best_estimator_.coef_.shape

(20, 3000)